In [1]:
import os
import json
import re
import faiss
import pickle
import gzip
import numpy as np
from tqdm import tqdm
import umap
import hdbscan
from hyperopt import hp
import chatintents
from chatintents import ChatIntents
from collections import Counter

import lib.search_utils as search

In [2]:
embs, dict_emb, faiss_index = search.build_vector_search_index(folder="./emb")
txts = [dict1["txt"] for i, dict1 in dict_emb.items()]

100%|██████████████████████████████████████████████████████████████████████████████| 2866/2866 [00:04<00:00, 609.81it/s]


In [3]:
umap_model = umap.UMAP(n_neighbors=15, n_components=5, metric='cosine')
umap_result = umap_model.fit_transform(embs)
clusters = hdbscan.HDBSCAN(min_cluster_size = 15, metric='euclidean', 
                       cluster_selection_method='eom').fit(umap_result)
cluster_labels = clusters.fit_predict(umap_result)

In [32]:
def chatgpt_summerize(txt):
    from lib.openai_utils import openai_client
    prompt = f"请给出以下文本的一个关键内容，不超过10个字。以下是文本：{txt}"

    response = openai_client.chat.completions.create(model="gpt-3.5-turbo-1106",
      messages=[{"role": "user", "content": prompt}])
    txt_response = json.loads(response.json())["choices"][0]["message"]["content"]
    pattern = r"关键词："
    txt_response = re.sub(pattern, "", txt_response)
    return txt_response


labels = [(k, v) for k, v in Counter(cluster_labels).items() if k!=-1]
labels = sorted(labels, key=lambda x:x[1], reverse=True)
for i, (label, c) in enumerate(labels):
    if i>10: break
    indices = np.where(cluster_labels==label)[0]
    centroid = np.mean(embs[indices], axis=0)
    centroid /= np.linalg.norm(centroid, ord=2, axis=-1, keepdims=True)
    D, I = faiss_index.search(centroid.reshape(1,-1), k=1)
    txt_centroid = "\n".join([txts[i] for i in I[0]])
    keyword = chatgpt_summerize(txt_centroid)
    print(keyword)

反对未经验证的疫苗
反共主张
音乐与心声
G-Fashion是价廉质优。
香港发生了重大事件
权力斗争 intensifies
郭文贵直播谈话


KeyboardInterrupt: 